<a href="https://colab.research.google.com/github/marcospiau/ia368-dd-dl4ir/blob/main/aula08-inpars/01_baseline_monot5_base_castorini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

I'm planning on trying to use the likelihood scoring evaluation already included in the T5 repository for re-ranking, instead of the Mesh-TensorFlow fork used by default in Pygaggle. If successful, it will be advantageous because it removes the need to maintain a fork of mesh-tensorflow, the default approach used by Pygaggle.

Before training any new models or developing new code using the new approach, we are going to validate the scoring and evaluation pipeline by modifying the original code from Castorini as little as possible.

# Hardware info

In [1]:
!echo *********
!echo lspu is "$(lscpu)"
# !echo *********
!echo free -mh is "$(free -mh)"
# !echo *********
!echo TPU_NAME is $TPU_NAME

sample_data
lspu is Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:                        0
CPU MHz:                         2199.998
BogoMIPS:                        4399.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        256 KiB
L3 cache:                        55 MiB
NUMA node0 CPU(s):             

# Gcloud authentication

In [2]:
import os
from google.colab import auth
import tensorflow_gcs_config

os.environ['USE_AUTH_EPHEM'] = '0'
auth.authenticate_user(clear_output=False)

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=AWeCFU2PoFcVo1yTLRrzdUgtM9JWPO&prompt=consent&access_type=offline&code_challenge=lr7sWftt_GYgwP3G3tcQ5JxrFoUz40_wajo7qODA7_Q&code_challenge_method=S256

Enter authorization code: 4/0AbUR2VN7p2j3kD97aZhXpP0JoaBMzcqlY5cl8BPtMpy6yoMbSOrKCRFPHAFH-HD3c73KsQ


In [3]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=ZHvWG8uMFIWbgp5qz7hgE04vf3IhJH&prompt=consent&access_type=offline&code_challenge=bIVB4ZZQLei4suVjp8T4kdp57J8K1MmS6Emf-k2utBs&code_challenge_method=S256

Enter authorization code: 4/0AbUR2VOyhMm_Zt2_5CXg-XPsO92MYuD039nZCUF4ob-VR4BVN4ywz6J1LluuXVye7_obNw

You are now logged in as [m136825@dac.unicamp.br].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [4]:
!gcloud auth application-default login


The environment variable [GOOGLE_APPLICATION_CREDENTIALS] is set to:
  [/content/.adc/adc.json]
Credentials will still be generated to the default location:
  [/content/.config/application_default_credentials.json]
To use these credentials, unset this environment variable before
running your application.

Do you want to continue (Y/n)?  Y

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=HxJU4P3ymcuK7maud2GqKGh0URBHg5&prompt=consent&access_type=offline&code_challenge=IxdE8Lc_c1-ghjzKQ_aSY2LWWSE1RTWJFGq1vgfSjLs&code_chal

In [37]:
BUCKET = 'gs://aula08-inpars'
os.environ['BUCKET'] = BUCKET
!gsutil ls {BUCKET}
!gsutil ls $BUCKET

# Installs

Anserini, pyserini and anserini-tools:

In [8]:
%%capture
!wget -nc https://raw.githubusercontent.com/marcospiau/ia368-dd-dl4ir/main/scripts/install_anserini.sh && chmod +x install_anserini.sh && time ./install_anserini.sh

Pygaggle, forked mesh-tensorflow and t5:

In [1]:
%%shell
#pip install cloud-tpu-client tensorflow==2.3.1 tensorflow-text t5==0.7.1
# mesh-tensorflow fork: needed only for this notebook
git clone https://github.com/castorini/mesh.git
pip install --editable mesh

# pygaggle
git clone --recursive https://github.com/castorini/pygaggle.git
cd pygaggle
pip install .

pip install -U t5[gcp,cache-tasks]==0.9.3
pip install -U jaxlib

Cloning into 'mesh'...
remote: Enumerating objects: 4040, done.
remote: Counting objects: 100% (483/483), done.
remote: Compressing objects: 100% (178/178), done.
remote: Total 4040 (delta 329), reused 379 (delta 303), pack-reused 3557
Receiving objects: 100% (4040/4040), 2.31 MiB | 10.96 MiB/s, done.
Resolving deltas: 100% (3060/3060), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mesh
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mesh-tensorflow
Cloning into 'pygaggle'...
remote: Enumerating objects: 1562, done.
remote: Counting objects: 100% (634/634), done.
remote: Compressing objects: 100% (232/232), done.
remote: Total 1562 (delta 528), reused 431 (delta 402), pack-reused 928
Receiving objects: 100% (1562/1562), 513.12 KiB | 1.45 MiB/s, done.
Resolving deltas: 100% (1001/1001), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 

Vanilla stuff:

In [13]:
%%capture
!pip install -q ftfy polars toolz cytoolz transformers datasets dm-tree
# !pip install -U t5[gcp,cache-tasks]==0.9.3
# !pip install -U jaxlib
!sudo apt install -qq tree htop

# Imports

In [70]:
import os
import pandas as pd
import polars as pl
import ftfy
import datasets

import functools

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

os.environ['POLARS_FMT_STR_LEN']='1000'

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

os.environ['POLARS_FMT_STR_LEN']='1000'

# Preparing TREC-COVID data

## Corpus

We load the corpus into a polars DataFrame. Texts are sorted by descending word count to reduce the amount of padding and to make eventual OOM erros occur early.

In [8]:
def read_hf_concat_title_text() -> pl.DataFrame:
    """Read corpus and concat title and text for indexing"""
    ds = datasets.load_dataset('BeIR/trec-covid', 'corpus')['corpus']
    df = pl.from_arrow(ds.data.table)
    df = df.with_columns(
        pl.format('{}. {}', 'title', 'text').alias('title_and_text'))
    df = df.select(pl.col('_id').alias('id'),
                   pl.col('title_and_text').alias('text'))
    # sort by number of words to reduce padding on indexing step
    # we puyt longer texts on beginning to make eventual OOM occur early
    # df = df.sort(pl.col('text').str.count_match(' '), descending=True)
    return df

df_corpus = read_hf_concat_title_text()
df_corpus.shape, df_corpus.head()

  0%|          | 0/1 [00:00<?, ?it/s]

((171332, 2),
 shape: (5, 2)
 ┌──────────┬───────────────────────────────────────────────────────────────────────────────────────┐
 │ id       ┆ text                                                                                  │
 │ ---      ┆ ---                                                                                   │
 │ str      ┆ str                                                                                   │
 ╞══════════╪═══════════════════════════════════════════════════════════════════════════════════════╡
 │ ug7v899j ┆ Clinical features of culture-proven Mycoplasma pneumoniae infections at King          │
 │          ┆ Abdulaziz University Hospital, Jeddah, Saudi Arabia. OBJECTIVE: This retrospective    │
 │          ┆ chart review describes the epidemiology and clinical features of 40 patients with     │
 │          ┆ culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University          │
 │          ┆ Hospital, Jeddah, Saudi Arabia. METHODS

Castorini dataprep script expects a tsv file:

In [9]:
df_corpus.write_csv('trec-covid-corpus.tsv', has_header=False, separator='\t')
!wc -l trec-covid-corpus.tsv
!ls -lht trec-covid-corpus.tsv
!head trec-covid-corpus.tsv

171332 trec-covid-corpus.tsv
-rw-r--r-- 1 root root 185M May  2 04:12 trec-covid-corpus.tsv
ug7v899j	Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of

## Queries

In [10]:
def convert_queries_to_trec_format(output_file):
    """Load queries and save to tsv"""
    ds = datasets.load_dataset('BeIR/trec-covid', 'queries')['queries']
    ds.to_csv(output_file, header=None, sep='\t', columns=['_id', 'text'])

QUERIES_FILE = 'trec-covid-queries_trec_format.txt'

convert_queries_to_trec_format(QUERIES_FILE)
!wc -l {QUERIES_FILE}
!cat {QUERIES_FILE}

Generating queries split: 0 examples [00:00, ? examples/s]

Dataset trec-covid downloaded and prepared to /root/.cache/huggingface/datasets/BeIR___trec-covid/queries/0.0.0/093f1fe2ffa7a9c72fa48239c8f279b51d6b171abd77737c7fd1406125307599. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

50 trec-covid-queries_trec_format.txt
1	what is the origin of COVID-19
2	how does the coronavirus respond to changes in the weather
3	will SARS-CoV2 infected people develop immunity? Is cross protection possible?
4	what causes death from Covid-19?
5	what drugs have been active against SARS-CoV or SARS-CoV-2 in animal studies?
6	what types of rapid testing for Covid-19 have been developed?
7	are there serological tests that detect antibodies to coronavirus?
8	how has lack of testing availability led to underreporting of true incidence of Covid-19?
9	how has COVID-19 affected Canada
10	has social distancing had an impact on slowing the spread of COVID-19?
11	what are the guidelines for triaging patients infected with coronavirus?
12	what are best practices in hospitals and at home in maintaining quarantine?
13	what are the transmission routes of coronavirus?
14	what evidence is there related to COVID-19 super spreaders
15	how long can the coronavirus live outside the body
16	how long doe

## Qrels (not needed for predictions)

In [11]:
def convert_qrels_to_trec_format(output_file):
    """Save trec-format qrels to file"""
    def to_line(ex):
        # qid 0 docid score
        return '{} 0 {} {}\n'.format(ex['query-id'], ex['corpus-id'],
                                   ex['score'])
    ds = datasets.load_dataset('BeIR/trec-covid-qrels')['test']
    with open(output_file, 'w') as f:
        for item in ds.to_list():
            line = to_line(item)
            f.write(line)

QRELS_FILE = 'trec-covid-qrels_trec_format.txt'

convert_qrels_to_trec_format(QRELS_FILE)
!wc -l {QRELS_FILE}
!head {QRELS_FILE}

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/BeIR___csv/BeIR--trec-covid-qrels-1766e3af5b0b856a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

66336 trec-covid-qrels_trec_format.txt
1 0 005b2j4b 2
1 0 00fmeepz 1
1 0 g7dhmyyo 2
1 0 0194oljo 1
1 0 021q9884 1
1 0 02f0opkr 1
1 0 047xpt2c 0
1 0 04ftw7k9 0
1 0 pl9ht0d0 0
1 0 05vx82oo 0


## Run

We will validate our pipeline using a BM25 run created on previous exercises to avoid duplicated work.

In [15]:
!gsutil cp -nc gs://marcospiau-doc2query-aula05/runs/run.title_and_text_no_expansion.txt .
!ls -lht run.title_and_text_no_expansion.txt
!wc -l run.title_and_text_no_expansion.txt
!head run.title_and_text_no_expansion.txt

Skipping existing item: file://./run.title_and_text_no_expansion.txt
-rw-r--r-- 1 root root 1.8M May  2 04:16 run.title_and_text_no_expansion.txt
50000 run.title_and_text_no_expansion.txt
1 Q0 dv9m19yk 1 7.285900 Anserini
1 Q0 0paafp5j 2 6.214600 Anserini
1 Q0 96zsd27n 3 6.214599 Anserini
1 Q0 hmvo5b0q 4 6.163500 Anserini
1 Q0 1ij25a7u 5 5.838400 Anserini
1 Q0 5d7zien3 6 5.726000 Anserini
1 Q0 xqqn1t4e 7 5.661700 Anserini
1 Q0 fqs40ivc 8 5.650400 Anserini
1 Q0 iohvj16d 9 5.638100 Anserini
1 Q0 dckuhrlf 10 5.628400 Anserini


The expected run format is "a TSV with <query_id>, <doc_id> and <rank>", so we will convert our data to this format:

In [29]:
def convert_run_to_castorini_msmarco_format(in_file, out_file):
    df = pl.read_csv(
        in_file,
        new_columns=['qid', 'q0', 'doc_id', 'rank', 'score', 'run_id'],
        separator=' ', has_header=False)
    df = df.select('qid', 'doc_id', 'rank')
    display(df.head())
    df.write_csv(out_file, separator='\t', has_header=False)


convert_run_to_castorini_msmarco_format(
    in_file='run.title_and_text_no_expansion.txt',
    out_file='run_bm25_msmarco_format.tsv')
!ls -lht run_bm25_msmarco_format.tsv
!wc -l run_bm25_msmarco_format.tsv
!head run_bm25_msmarco_format.tsv

qid,doc_id,rank
i64,str,i64
1,"""dv9m19yk""",1
1,"""0paafp5j""",2
1,"""96zsd27n""",3
1,"""hmvo5b0q""",4
1,"""1ij25a7u""",5


-rw-r--r-- 1 root root 768K May  2 04:23 run_bm25_msmarco_format.tsv
50000 run_bm25_msmarco_format.tsv
1	dv9m19yk	1
1	0paafp5j	2
1	96zsd27n	3
1	hmvo5b0q	4
1	1ij25a7u	5
1	5d7zien3	6
1	xqqn1t4e	7
1	fqs40ivc	8
1	iohvj16d	9
1	dckuhrlf	10


## Creating IDs and text files from BM25 run

In [12]:
!python3 pygaggle/pygaggle/data/create_msmarco_monot5_input.py --help

usage: create_msmarco_monot5_input.py
       [-h]
       --queries
       QUERIES
       --run
       RUN
       --corpus
       CORPUS
       --t5_input
       T5_INPUT
       --t5_input_ids
       T5_INPUT_IDS

options:
  -h, --help
    show this
    help
    message and
    exit
  --queries QUERIES
    tsv file
    with two
    columns,
    <query_id>
    and <query_
    text>
  --run RUN
    tsv file
    with three
    columns
    <query_id>,
    <doc_id>
    and <rank>
  --corpus CORPUS
  --t5_input T5_INPUT
    path to
    store
    t5_input,
    txt format
  --t5_input_ids T5_INPUT_IDS
    path to
    store the
    query-doc
    ids of
    t5_input,
    tsv format


In [31]:
!time python3 pygaggle/pygaggle/data/create_msmarco_monot5_input.py \
--queries=trec-covid-queries_trec_format.txt \
--run=run_bm25_msmarco_format.tsv \
--corpus=trec-covid-corpus.tsv \
--t5_input=castorini-baseline-t5-input.txt \
--t5_input_ids=castorini-baseline-t5-input_ids.txt

Loading corpus...
171332it [00:01, 153369.37it/s]
Loading queries...
50it [00:00, 475544.67it/s]
Loading run...
50000it [00:00, 750089.24it/s]
Sorting candidate docs by rank...
100% 50/50 [00:00<00:00, 7679.34it/s]
Writing t5 input and ids
100% 50/50 [00:00<00:00, 128.42it/s]

real	0m1.813s
user	0m1.099s
sys	0m0.628s


In [35]:
!ls -lth castorini-baseline-t5-input*
!wc -l castorini-baseline-t5-input*
!head castorini-baseline-t5-input*

-rw-r--r-- 1 root root 578K May  2 04:26 castorini-baseline-t5-input_ids.txt
-rw-r--r-- 1 root root  82M May  2 04:26 castorini-baseline-t5-input.txt
   50000 castorini-baseline-t5-input_ids.txt
   50000 castorini-baseline-t5-input.txt
  100000 total
==> castorini-baseline-t5-input_ids.txt <==
1	dv9m19yk
1	0paafp5j
1	96zsd27n
1	hmvo5b0q
1	1ij25a7u
1	5d7zien3
1	xqqn1t4e
1	fqs40ivc
1	iohvj16d
1	dckuhrlf

==> castorini-baseline-t5-input.txt <==
Query: what is the origin of COVID-19 Document: [What is the origin of SARS-CoV-2?]. Every time a pandemic occurs, dozens of theories emerge to attribute the origin of the event to different facts. The COVID-19 pandemic that has hit virtually all the globe has been no exception. What is known so far about the origin of the virus that causes COVID 19? The first investigations on the origin of this disease have determined that it is a new type of virus, the origin of which is most likely zoonotic. Relevant:
Query: what is the origin of COVID-19 Docum

Sending data to GCP (required for using T5 with TPUs):

In [40]:
!ls -lht

total 271M
-rw-r--r--  1 root root  578K May  2 04:26 castorini-baseline-t5-input_ids.txt
-rw-r--r--  1 root root   82M May  2 04:26 castorini-baseline-t5-input.txt
-rw-r--r--  1 root root  768K May  2 04:23 run_bm25_msmarco_format.tsv
-rw-r--r--  1 root root  1.8M May  2 04:16 run.title_and_text_no_expansion.txt
-rw-r--r--  1 root root 1023K May  2 04:14 trec-covid-qrels_trec_format.txt
-rw-r--r--  1 root root  3.6K May  2 04:13 trec-covid-queries_trec_format.txt
-rw-r--r--  1 root root  185M May  2 04:12 trec-covid-corpus.tsv
drwxr-xr-x 15 root root  4.0K May  2 03:49 pygaggle
drwxr-xr-x  9 root root  4.0K May  2 03:49 mesh
drwxr-xr-x 13 root root  4.0K May  2 03:45 anserini
drwxr-xr-x 15 root root  4.0K May  2 03:44 pyserini
drwxr-xr-x  6 root root  4.0K May  2 03:39 tools
-rwxr-xr-x  1 root root  1011 May  2 03:38 install_anserini.sh
drwxr-xr-x  1 root root  4.0K Apr 28 13:35 sample_data


In [44]:
!gsutil -m cp castorini-baseline-t5-input_ids.txt \
castorini-baseline-t5-input.txt \
run.title_and_text_no_expansion.txt \
trec-covid-qrels_trec_format.txt \
trec-covid-queries_trec_format.txt \
trec-covid-corpus.tsv \
$BUCKET/castorini_baseline

Copying file://trec-covid-corpus.tsv [Content-Type=text/tab-separated-values]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://trec-covid-qrels_trec_format.txt [Content-Type=text/plain]...
Copying file://run.title_and_text_no_expansion.txt [Content-Type=text/plain]...
Copying file://castorini-baseline-t5-input.txt [Content-Type=text/plain]...
Copying fi

# Generating predictions with castorini monot5-base

The script used here is basically the same as described on https://github.com/castorini/pygaggle/blob/master/docs/experiments-robust04-monot5-tpu.md.

In [61]:
%%writefile monot5-base-castorini-baseline.sh
set -u
export MODEL_NAME=base
export MODEL_DIR=gs://castorini/monot5/experiments/${MODEL_NAME}
export CHECKPOINT_STEP=1100000

ZONE=us-central1-f
TPU_SIZE=v2-8

INPUT_FILENAME=$1
OUTPUT_FILENAME=$2
LOG_FILE=$3


# --gcp_project="${PROJECT_NAME}" \

time python3 -m t5.models.mesh_transformer_main \
--gin_file="gs://t5-data/pretrained_models/base/operative_config.gin" \
--tpu="${TPU_NAME}" \
--tpu_zone="${ZONE}" \
--model_dir="${MODEL_DIR}" \
--gin_param="utils.tpu_mesh_shape.tpu_topology = '${TPU_SIZE}'" \
--gin_file="infer.gin" \
--gin_file="beam_search.gin" \
--gin_param="infer_checkpoint_step = ${CHECKPOINT_STEP}" \
--gin_param="utils.run.sequence_length = {'inputs': 512, 'targets': 2}" \
--gin_param="Bitransformer.decode.max_decode_length = 2" \
--gin_param="input_filename = '${INPUT_FILENAME}'" \
--gin_param="output_filename = '${OUTPUT_FILENAME}'" \
--gin_param="utils.run.batch_size=('tokens_per_batch', 65536)" \
--gin_param="Bitransformer.decode.beam_size = 1" \
--gin_param="Bitransformer.decode.temperature = 0.0" \
--gin_param="Unitransformer.sample_autoregressive.sampling_keep_top_k = -1" \
2>&1 | tee -a $LOG_FILE

Overwriting monot5-base-castorini-baseline.sh


In [62]:
!chmod +x monot5-base-castorini-baseline.sh && \
time ./monot5-base-castorini-baseline.sh \
gs://aula08-inpars/castorini_baseline/castorini-baseline-t5-input.txt \
gs://aula08-inpars/castorini_baseline/castorini-baseline-monot5-base-predictions.txt \
castorini-baseline-monot5-base-predictions.log

2023-05-02 04:49:11.950094: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Instructions for updating:
non-resource variables are not supported in the long term
I0502 04:49:24.268564 140415953385280 mesh_transformer_main.py:171] No write access to model directory. Skipping command logging.
I0502 04:49:24.274960 140415953385280 resource_reader.py:50] system_path_file_exists:gs://t5-data/pretrained_models/base/operative_config.gin
E0502 04:49:24.275991 140415953385280 resource_reader.py:55] Path not found: gs://t5-data/pretrained_models/base/operative_config.gin
I0502 04:49:24.411886 140415953385280 config.py:2372] Skipping import of unknown module `t5.data.sentencepiece_vocabulary` (skip_unknown=True).
I0502 04:49:24.429738 140415953385280 resource_reader.py:50] system_path_file_exists:infer.gin
E0502 04:49:24.430242 140415953385280 resource_reader.py:55] Path not found: infer.gin
I0502 04:49:24.434230 140415953385280 resource_reader.py:50

It took 3m58.348s to score 50_000 pairs of queries and documents.

Copying data to local directory:

In [76]:
!gsutil cp -nc gs://aula08-inpars/castorini_baseline/castorini-baseline-monot5-base-predictions.txt-1100000 .
!ls -lht castorini-baseline-monot5-base-predictions.txt-1100000
!wc -l castorini-baseline-monot5-base-predictions.txt-1100000
!head castorini-baseline-monot5-base-predictions.txt-1100000


Copying gs://aula08-inpars/castorini_baseline/castorini-baseline-monot5-base-predictions.txt-1100000...
/ [1 files][  1.2 MiB/  1.2 MiB]                                                
Operation completed over 1 objects/1.2 MiB.                                      
-rw-r--r-- 1 root root 1.3M May  2 05:00 castorini-baseline-monot5-base-predictions.txt-1100000
50000 castorini-baseline-monot5-base-predictions.txt-1100000
true	-0.00809466510988807
true	-0.0049174766998860125
true	-0.0049174766998860125
true	-0.03212943377211589
true	-0.01814992791780978
true	-0.00975591100022136
true	-0.006715348489117967
true	-0.013311021623458399
true	-0.0028048569030831336
true	-0.006715348489117967


# Evaluating results

In [71]:
import shlex
import subprocess

TREC_EVAL_BIN_PATH = './tools/eval/trec_eval.9.0.4/trec_eval'
# Essa função foi escrita usando o github copilot
def get_trec_eval_metrics(flags, qrels_path, results_path):
    """Runs trec_eval and returns the results as a dictionary.

    Args:
        flags (str): Flags to pass to trec_eval.
        qrels_path (str): Path to the qrels file.
        results_path (str): Path to the results file.

    Returns:
        Dict[str, float]: A dictionary mapping metric names to their values.
    """
    output = subprocess.check_output([
        TREC_EVAL_BIN_PATH,
        qrels_path,
        results_path,
        *shlex.split(flags)
    ]).decode('utf-8')
    return {
        line.split()[0]: (line.split()[2])
        for line in output.splitlines()
    }


eval_metrics_fn = functools.partial(
    get_trec_eval_metrics,
    '-c -m ndcg_cut.10 -m recall.1000 -m recip_rank.10',
    'trec-covid-qrels_trec_format.txt'
)

Before evaluating the results, we need to convert our predictions to 'trec-format':

In [74]:
!python3 pygaggle/pygaggle/data/convert_run_from_t5_to_trec_format.py --help

usage: convert_run_from_t5_to_trec_format.py
       [-h]
       --predictions
       PREDICTIONS
       --query_run_ids
       QUERY_RUN_IDS
       --output
       OUTPUT

Convert T5
predictions
into a
TREC-
formatted
run.

options:
  -h, --help
    show this
    help
    message and
    exit
  --predictions PREDICTIONS
    T5
    predictions
    file.
  --query_run_ids QUERY_RUN_IDS
    File
    containing
    query doc
    id pairs
    paired with
    the T5's
    predictions
    file.
  --output OUTPUT
    run file in
    the TREC
    format.


In [80]:
!python3 pygaggle/pygaggle/data/convert_run_from_t5_to_trec_format.py \
    --predictions=castorini-baseline-monot5-base-predictions.txt-1100000 \
    --query_run_ids=castorini-baseline-t5-input_ids.txt \
    --output=castorini-baseline-monot5-base-trec-format-run.txt

!ls -lht
!wc -l castorini-baseline-monot5-base-trec-format-run.txt
!head castorini-baseline-monot5-base-trec-format-run.txt

Done!
total 274M
-rw-r--r--  1 root root  2.1M May  2 05:06 castorini-baseline-monot5-base-trec-format-run.txt
-rw-r--r--  1 root root  1.3M May  2 05:00 castorini-baseline-monot5-base-predictions.txt-1100000
-rw-r--r--  1 root root  296K May  2 04:53 castorini-baseline-monot5-base-predictions.log
-rwxr-xr-x  1 root root  1.2K May  2 04:48 monot5-base-castorini-baseline.sh
-rw-r--r--  1 root root  578K May  2 04:26 castorini-baseline-t5-input_ids.txt
-rw-r--r--  1 root root   82M May  2 04:26 castorini-baseline-t5-input.txt
-rw-r--r--  1 root root  768K May  2 04:23 run_bm25_msmarco_format.tsv
-rw-r--r--  1 root root  1.8M May  2 04:16 run.title_and_text_no_expansion.txt
-rw-r--r--  1 root root 1023K May  2 04:14 trec-covid-qrels_trec_format.txt
-rw-r--r--  1 root root  3.6K May  2 04:13 trec-covid-queries_trec_format.txt
-rw-r--r--  1 root root  185M May  2 04:12 trec-covid-corpus.tsv
drwxr-xr-x 15 root root  4.0K May  2 03:49 pygaggle
drwxr-xr-x  9 root root  4.0K May  2 03:49 mesh
d

In [82]:
eval_metrics_fn('castorini-baseline-monot5-base-trec-format-run.txt')

{'recip_rank': '0.8607', 'recall_1000': '0.3955', 'ndcg_cut_10': '0.7178'}

BM25 with no re-ranking:

In [83]:
eval_metrics_fn('run.title_and_text_no_expansion.txt')

{'recip_rank': '0.8529', 'recall_1000': '0.3955', 'ndcg_cut_10': '0.5947'}